<a href="https://colab.research.google.com/github/elaysason/Crypto-Exp3/blob/main/lemida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime
import random
from operator import itemgetter
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import defaultdict

In [ ]:
coins_list = ["Aave", "BinanceCoin", "Bitcoin", "Cardano", "ChainLink", "Cosmos", "CryptocomCoin", "Dogecoin", "EOS",
              "Ethereum", "Iota", "Litecoin", "Monero", "NEM", "Polkadot", "Solana", "Stellar", "Tether",
              "Tron", "Uniswap", "USDCoin", "WrappedBitcoin", "XRP"]
K = 23
epsilon = [1 / K]
t = 0
T = 2991

reward_sum = dict()


In [ ]:
crypto_datasets = dict()
for c in coins_list:
    file = "coin_" + c + ".csv"
    crypto = pd.read_csv(file)
    crypto['Date'] = pd.to_datetime(crypto['Date']).dt.date
    crypto_datasets[c] = crypto
    reward_sum[c] = 0

In [ ]:
def get_date(start_date, days):
    return start_date + datetime.timedelta(days=days)

In [ ]:
def payoff(coin, start_date, t, amountToInvest=1.0):
    date = get_date(start_date, t).date()
    on_date = crypto_datasets[coin].loc[crypto_datasets[coin]["Date"] == date]
    if len(on_date) == 0:
        before_date = crypto_datasets[coin].loc[crypto_datasets[coin]["Date"] < date].sort_values(by=["Date"],ascending=False)
        open = before_date['Open'].head(20).mean()
        close = before_date['Close'].head(20).mean()
        return amountToInvest*close/open - amountToInvest

    sharesBought = amountToInvest / pd.to_numeric(on_date['Open'])
    amountAfterSale = list(pd.to_numeric(sharesBought) * pd.to_numeric(on_date['Close']))[0]

    return amountAfterSale - amountToInvest

In [ ]:
def get_existing_coins(start_date, days):
    existing = []
    new = []
    for i, key in enumerate(crypto_datasets.keys()):
        if min(crypto_datasets[key]["Date"]) <= get_date(start_date, days).date():
            existing.append(i)
    return existing

In [ ]:
def choose_coin(coin_value_list):
    coins = [tup[0] for tup in coin_value_list]
    probs = [float(tup[1]) for tup in coin_value_list]
    return random.choices(coins, weights=probs)[0]

In [ ]:
def exp3_base(date, days):
    eta = np.sqrt(np.log(K) / (days* K))
    start_date = datetime.datetime.strptime(date, "%Y-%m-%d")
    existing_coins = get_existing_coins(start_date, 0)
    scores = [0] * K
    reward_sum = 0
    for t in range(1, days):
        cur_distribution = []
        sum_scores = np.sum([np.exp(scores[j] * eta) for j in existing_coins])
        for i in range(K):
            if i in existing_coins:
                cur_distribution.append(
                    np.exp(eta * scores[i]) / sum_scores)
            else:
                cur_distribution.append(0)
        coins_values = [(coin, cur_distribution[coin]) for coin in existing_coins]
        chosen_coin = choose_coin(coins_values)
        cur_reward = payoff(coins_list[chosen_coin], start_date, t)
        for i in range(K):
            if i in existing_coins:
                scores[i] = scores[i] + 1
                if chosen_coin == i:
                    scores[i] -= (1 - cur_reward) / cur_distribution[i]
        reward_sum += cur_reward * cur_distribution[chosen_coin]
        existing_coins = get_existing_coins(start_date, t)
    return reward_sum


In [ ]:
def exp3_extension1(date, days):
    epsilon = [1 / K]
    start_date = datetime.datetime.strptime(date, "%Y-%m-%d")
    rewards = []
    existing_coins = get_existing_coins(start_date, 0)
    scores = [0] * K
    reward_sum = 0
    for t in range(1, days):
        epsilon.append(min([epsilon[0], math.sqrt(np.log(len(existing_coins)) / (len(existing_coins) * t))]))
        cur_distribution = []
        sum_scores = sum([math.exp(epsilon[t - 1] * scores[c]) for c in existing_coins])
        for i in range(K):
            if i in existing_coins:
                cur_distribution.append(1 - len(existing_coins) * epsilon[t])
                val = math.exp(epsilon[t - 1] * scores[i]) / sum_scores
                cur_distribution[i] *= val
                cur_distribution[i] += epsilon[t]
            else:
                cur_distribution.append(0)
        coins_values = [(coin, cur_distribution[coin]) for coin in existing_coins]
        chosen_coin = choose_coin(coins_values)
        cur_reward = payoff(coins_list[chosen_coin], start_date, t)
        scores[chosen_coin] += (cur_reward) / cur_distribution[chosen_coin]
        reward_sum += cur_reward * cur_distribution[chosen_coin]
        existing_coins = get_existing_coins(start_date, t)
    return reward_sum


In [ ]:
def payoff_to_day(coin,start_date,day):
    return sum([payoff(coin,start_date,i) for i in range(day)])

In [ ]:
def get_best_coin_sum(date, days):
    start_date = datetime.datetime.strptime(date, "%Y-%m-%d")
    best_sum = -float("inf")
    best_coin = list(crypto_datasets.keys())[0]
    for coin in crypto_datasets.keys():
        reward_sum = 0
        for t in range(days):
            reward_sum += payoff(coin, start_date, t)
        if reward_sum >= best_sum:
            best_coin = coin
            best_sum = reward_sum

In [ ]:
rewards_extended = [exp3_extension1('2013-04-29', i) for i in days]
rewards_basis = [exp3_base('2013-04-29', i) for i in days]
best_coin_rewards = [get_best_coin_sum("2013-04-29", i) for i in days]
best_coin = best_coin_rewards[-1][0]
best_rewards = [r[1] for r in best_coin_rewards]
regrets_extended = [best_rewards[i] - rewards_extended[i] for i in range(len(days))]
regrets_basis = [best_rewards[i] - rewards_basis[i] for i in range(len(days))]

In [ ]:
days = range(1, 2900, 10)
regrets = [exp3_base("2013-04-29", t) for t in days]
plt.title('Regret As a Function of Number of Days')
plt.plot(days,[np.sqrt(np.log(K) * (day* K)) for day in days],label='Bound')
plt.plot(days, regrets_basis, label='Basis Algorithm')
plt.xlabel('Number Of Days')
plt.ylabel('Regret')
plt.legend()
plt.show()

In [ ]:
days = range(600, 2000, 50)
rewards_extended= [exp3_extension1('2013-04-29',i) for i in days]
rewards_basis = [exp3_base('2013-04-29',i) for i in days]
best_coin_rewards = [get_best_coin_sum("2013-04-29", i) for i in days]
best_coin = best_coin_rewards[-1][0]
best_rewards = [r[1] for r in best_coin_rewards]
regrets_extended = [best_rewards[i] - rewards_extended[i] for i in days]
regrets_basis = [best_rewards[i] - rewards_basis[i] for i in days]

plt.title('Regret As a Function of Number of Days')
plt.plot(days, regrets_extended, label = 'Extented 1')
plt.plot(days, regrets_basis,label = 'Basis')
plt.xlabel('Number Of Days')
plt.ylabel('Regret')
plt.show()

In [ ]:
plt.title('Regret As a Function of Number of Days')
plt.plot(days, regrets_basis, label='First Extension')
plt.plot(days, regrets_extended, label='Basis')
plt.xlabel('Number Of Days')
plt.legend()
plt.ylabel('Regret')
plt.show()